## 📈 Predicting Premier League Final Positions Using Betting Odds & Simulation

**Competition:** English Premier League 2025/26  
**Purpose:** Estimate probabilities of final league positions using betting market information and simulation  
**Methods:** Odds-implied probabilities, Monte Carlo simulation, scenario analysis  
**Author:** [Victoria Friss de Kereki](https://www.linkedin.com/in/victoria-friss-de-kereki/)  

---

**Notebook first written:** `17/01/2026`  
**Last updated:** `17/01/2026`  

> This notebook develops a probabilistic framework to predict final Premier League final positions using betting odds as market-based expectations.
>
> Betting odds are transformed into implied probabilities and adjusted for bookmaker margin. These probabilities are then used to simulate the remainder of the season via Monte Carlo methods, generating distributions over final points totals and league positions.
>
> The analysis focuses on estimating the likelihood of key outcomes such as title wins, top-four finishes, relegation, and mid-table placements. Results are presented at team level with uncertainty intervals, and the framework can be extended to incorporate form, fixture difficulty, or alternative predictive inputs beyond betting markets.


In [157]:
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import requests
from scipy.stats import poisson
from dotenv import load_dotenv
import soccerdata as sd
import matplotlib.pyplot as plt

## 1. Premier League Final Standings (ESPN Scraping)
##### Using the ESPN scraper I built in my previous project.

In [158]:
year = 2025  # current Premier League season start year

url = f"https://www.espn.com/soccer/standings/_/league/ENG.1/season/{year}"
tables = pd.read_html(url)

teams_raw = tables[0]
stats = tables[1]

teams = pd.DataFrame()
teams["position"] = teams_raw.iloc[:, 0].str.extract(r"^(\d+)").astype(int)
teams["team"] = (
    teams_raw.iloc[:, 0]
    .str.replace(r"^\d+", "", regex=True)
    .str.replace(r"^[A-Z]{2,3}", "", regex=True)
    .str.strip()
)

stats.columns = ["gp", "w", "d", "l", "gf", "ga", "gd", "pts"]
stats = stats.apply(lambda c: c.astype(str)
                              .str.replace("+", "", regex=False)
                              .astype(int))

premierleague = pd.concat([teams, stats], axis=1)
# premierleague["season"] = f"{year}-{year+1}"

premierleague


,position,team,gp,w,d,l,gf,ga,gd,pts
0,1,Arsenal,22,15,5,2,40,14,26,50
1,2,Manchester City,22,13,4,5,45,21,24,43
2,3,Aston Villa,22,13,4,5,33,25,8,43
3,4,Liverpool,22,10,6,6,33,29,4,36
4,5,Manchester United,22,9,8,5,38,32,6,35
5,6,Chelsea,22,9,7,6,36,24,12,34
6,7,Brentford,22,10,3,9,35,30,5,33
7,8,Newcastle United,22,9,6,7,32,27,5,33
8,9,Sunderland,22,8,9,5,23,23,0,33
9,10,Everton,22,9,5,8,24,25,-1,32


## 2. Get betting odds using API

In [159]:
# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("ODDS_DATA_API_KEY")

if API_KEY is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [160]:
url = "https://api.the-odds-api.com/v4/sports/soccer_epl/odds"

params = {
    "apiKey": API_KEY,
    "regions": "uk",
    "markets": "h2h",
    "oddsFormat": "decimal",
    "dateFormat": "iso",
    "days": 365  # get all upcoming matches for the next year
}

response = requests.get(url, params=params)
response.raise_for_status()

odds_data = response.json()
print("Total upcoming matches:", len(odds_data))

Total upcoming matches: 20


In [161]:
def flatten_odds(data):
    rows = []

    for match in data:
        match_id = match["id"]
        home = match["home_team"]
        away = match["away_team"]
        time = match["commence_time"]

        for book in match["bookmakers"]:
            bookmaker = book["title"]

            # Find head-to-head (h2h) market. Find the market where key == 'h2h' (win/draw/win odds). If not found, skip this bookmaker.
            h2h = next((m for m in book["markets"] if m["key"] == "h2h"), None)
            if not h2h:
                continue

            outcomes = {o["name"]: o["price"] for o in h2h["outcomes"]}

            rows.append({
                "match_id": match_id,
                "commence_time": time,
                "home_team": home,
                "away_team": away,
                "bookmaker": bookmaker,
                "home_odds": outcomes.get(home),
                "draw_odds": outcomes.get("Draw"),
                "away_odds": outcomes.get(away),
            })

    return pd.DataFrame(rows)

df = flatten_odds(odds_data)
df.head()

,match_id,commence_time,home_team,away_team,bookmaker,home_odds,draw_odds,away_odds
0,a396966205b21287109ea8cae42e44c1,2026-01-24T12:30:00Z,West Ham United,Sunderland,Unibet (UK),2.45,3.30,2.88
1,a396966205b21287109ea8cae42e44c1,2026-01-24T12:30:00Z,West Ham United,Sunderland,Paddy Power,2.45,3.20,2.88
2,a396966205b21287109ea8cae42e44c1,2026-01-24T12:30:00Z,West Ham United,Sunderland,Betway,2.50,3.20,2.80
3,a396966205b21287109ea8cae42e44c1,2026-01-24T12:30:00Z,West Ham United,Sunderland,Smarkets,2.56,3.35,2.98
4,a396966205b21287109ea8cae42e44c1,2026-01-24T12:30:00Z,West Ham United,Sunderland,Sky Bet,2.45,3.20,2.80


In [162]:
betting_odds_avg = (
    df.groupby(["match_id", "home_team", "away_team"])
      .agg({
          "home_odds": "mean",
          "draw_odds": "mean",
          "away_odds": "mean"
      })
      .reset_index()
)

betting_odds_avg.head()

,match_id,home_team,away_team,home_odds,draw_odds,away_odds
0,1ca6d3d9cde3e58a39211feb9188530c,Newcastle United,Aston Villa,1.953333,3.666667,3.563889
1,1e811fa7ead0a3e6ef920b15b2bbb95d,Burnley,Tottenham Hotspur,3.675000,3.386111,2.021667
2,36820753efb36739a83c6e5e440827b2,Brighton and Hove Albion,Everton,1.820714,3.621429,4.130714
3,38a3cb5e295f55e274d589fc646cf2dd,Tottenham Hotspur,Manchester City,4.418571,3.946429,1.690000
4,4d6c0ac53b2b38aeb5bc8c8163f3f8b9,Crystal Palace,Chelsea,3.455556,3.622222,2.014444


In [163]:
# Convert odds -> raw probabilities
betting_odds_avg["p_home_raw"] = 1 / betting_odds_avg["home_odds"]
betting_odds_avg["p_draw_raw"] = 1 / betting_odds_avg["draw_odds"]
betting_odds_avg["p_away_raw"] = 1 / betting_odds_avg["away_odds"]

#2) Normalize (remove bookmaker margin)
betting_odds_avg["total_raw"] = (
    betting_odds_avg["p_home_raw"] +
    betting_odds_avg["p_draw_raw"] +
    betting_odds_avg["p_away_raw"]
)

betting_odds_avg["p_home_book"] = betting_odds_avg["p_home_raw"] / betting_odds_avg["total_raw"]
betting_odds_avg["p_draw_book"] = betting_odds_avg["p_draw_raw"] / betting_odds_avg["total_raw"]
betting_odds_avg["p_away_book"] = betting_odds_avg["p_away_raw"] / betting_odds_avg["total_raw"]

In [164]:
# Keep only useful columns
betting_odds_avg = betting_odds_avg[[
#match_id",
    "home_team",
    "away_team",
    "p_home_book",
    "p_draw_book",
    "p_away_book"
]]

betting_odds_avg.head()

,home_team,away_team,p_home_book,p_draw_book,p_away_book
0,Newcastle United,Aston Villa,0.480580,0.256018,0.263401
1,Burnley,Tottenham Hotspur,0.256205,0.278063,0.465731
2,Brighton and Hove Albion,Everton,0.514526,0.258684,0.226790
3,Tottenham Hotspur,Manchester City,0.211230,0.236501,0.552269
4,Crystal Palace,Chelsea,0.272526,0.259986,0.467488


## 3. Get fixtures for upcoming EPL games

In [165]:
# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("FOOTBALL_DATA_API_KEY")

if API_KEY is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [166]:
url = "https://api.football-data.org/v4/competitions/PL/matches"

headers = {
    "X-Auth-Token": API_KEY
}

today = datetime.utcnow().date()
end_of_season = today + timedelta(days=365)  # big range to cover all remaining games

params = {
    "status": "SCHEDULED",
    "dateFrom": today.isoformat(),
    "dateTo": end_of_season.isoformat()
}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()

data = response.json()
fixtures = data["matches"]

df_fixtures = pd.DataFrame(fixtures)

df_fixtures_clean = df_fixtures[[
    "utcDate",
    "status",
    "homeTeam",
    "awayTeam"
]]

df_fixtures_clean.head()
print("Total scheduled matches:", len(df_fixtures_clean))


[01/19/26 20:49:36] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_21604\187534790.py:7:      ]8;id=303943;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=129471;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             DeprecationWarning: datetime.datetime.utcnow() is deprecated and                      
                             scheduled for removal in a future version. Use timezone-aware objects                 
                             to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).                   
                               today = datetime.utcnow().date()                                                    
                                                                                                                   

Total scheduled matches: 160


In [167]:
df_fixtures_clean["homeTeam"] = df_fixtures_clean["homeTeam"].apply(lambda x: x["name"])
df_fixtures_clean["awayTeam"] = df_fixtures_clean["awayTeam"].apply(lambda x: x["name"])

[01/19/26 20:49:38] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_21604\2614041168.py:1:     ]8;id=289672;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=844357;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_fixtures_clean["homeTeam"] =                                                     
                             df_fixtures_clean["homeTeam"].apply(lambda x: x["name"])                              
                                                                                                                   

                    WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_21604\2614041168.py:2:     ]8;id=652050;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=404658;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_fixtures_clean["awayTeam"] =                                                     
                             df_fixtures_clean["awayTeam"].apply(lambda x: x["name"])                              
                                                                                                                   

In [168]:
df_fixtures_clean

,utcDate,status,homeTeam,awayTeam
0,2026-01-24T12:30:00Z,TIMED,West Ham United FC,Sunderland AFC
1,2026-01-24T15:00:00Z,TIMED,Burnley FC,Tottenham Hotspur FC
2,2026-01-24T15:00:00Z,TIMED,Fulham FC,Brighton & Hove Albion FC
3,2026-01-24T15:00:00Z,TIMED,Manchester City FC,Wolverhampton Wanderers FC
4,2026-01-24T17:30:00Z,TIMED,AFC Bournemouth,Liverpool FC
...,...,...,...,...
155,2026-05-24T15:00:00Z,TIMED,Liverpool FC,Brentford FC
156,2026-05-24T15:00:00Z,TIMED,Manchester City FC,Aston Villa FC
157,2026-05-24T15:00:00Z,TIMED,Nottingham Forest FC,AFC Bournemouth
158,2026-05-24T15:00:00Z,TIMED,Tottenham Hotspur FC,Everton FC


## 4. Get this season (2025/26) and last season (2024/25) results

In [169]:
url = "https://api.football-data.org/v4/competitions/PL/matches"
params = {
    "season": 2025,   # season year
    "status": "FINISHED"
}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
past_matches = response.json()["matches"]

In [170]:
clean_rows = []

for m in past_matches:
    row = {
        "utcDate": m["utcDate"],
        "matchday": m["matchday"],
        "status": m["status"],
        "homeTeam": m["homeTeam"]["name"],
        "awayTeam": m["awayTeam"]["name"],
        "homeGoals": m["score"]["fullTime"]["home"],
        "awayGoals": m["score"]["fullTime"]["away"],
        "winner": m["score"]["winner"]
    }
    clean_rows.append(row)

past_matches_25_clean = pd.DataFrame(clean_rows)
past_matches_25_clean.tail()

,utcDate,matchday,status,homeTeam,awayTeam,homeGoals,awayGoals,winner
215,2026-01-17T15:00:00Z,22,FINISHED,Tottenham Hotspur FC,West Ham United FC,1,2,AWAY_TEAM
216,2026-01-17T17:30:00Z,22,FINISHED,Nottingham Forest FC,Arsenal FC,0,0,DRAW
217,2026-01-18T14:00:00Z,22,FINISHED,Wolverhampton Wanderers FC,Newcastle United FC,0,0,DRAW
218,2026-01-18T16:30:00Z,22,FINISHED,Aston Villa FC,Everton FC,0,1,AWAY_TEAM
219,2026-01-19T20:00:00Z,22,FINISHED,Brighton & Hove Albion FC,AFC Bournemouth,1,1,DRAW


In [171]:
url = "https://api.football-data.org/v4/competitions/PL/matches"
params = {
    "season": 2024,   # season year
    "status": "FINISHED"
}

response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
past_matches_24 = response.json()["matches"]

In [172]:
clean_rows = []

for m in past_matches_24:
    row = {
        "utcDate": m["utcDate"],
        "matchday": m["matchday"],
        "status": m["status"],
        "homeTeam": m["homeTeam"]["name"],
        "awayTeam": m["awayTeam"]["name"],
        "homeGoals": m["score"]["fullTime"]["home"],
        "awayGoals": m["score"]["fullTime"]["away"],
        "winner": m["score"]["winner"]
    }
    clean_rows.append(row)

past_matches_24_clean = pd.DataFrame(clean_rows)
past_matches_24_clean.head()

,utcDate,matchday,status,homeTeam,awayTeam,homeGoals,awayGoals,winner
0,2024-08-16T19:00:00Z,1,FINISHED,Manchester United FC,Fulham FC,1,0,HOME_TEAM
1,2024-08-17T11:30:00Z,1,FINISHED,Ipswich Town FC,Liverpool FC,0,2,AWAY_TEAM
2,2024-08-17T14:00:00Z,1,FINISHED,Arsenal FC,Wolverhampton Wanderers FC,2,0,HOME_TEAM
3,2024-08-17T14:00:00Z,1,FINISHED,Everton FC,Brighton & Hove Albion FC,0,3,AWAY_TEAM
4,2024-08-17T14:00:00Z,1,FINISHED,Newcastle United FC,Southampton FC,1,0,HOME_TEAM


## 5. Combine and calculate probabilities of W/D/L for each match

In [173]:
# Load Dataframes
df_current = past_matches_25_clean
df_prev = past_matches_24_clean
df_future = df_fixtures_clean

# Combine all past fixtures together
df_all = pd.concat([df_prev, df_current], ignore_index=True)

In [174]:
# Add weights: more recent games = more weight
df_all["date"] = pd.to_datetime(df_all["utcDate"])
df_all["weight"] = np.linspace(1, 2, len(df_all))  # simple linear weighting

In [175]:
df_all.tail()

,utcDate,matchday,status,homeTeam,awayTeam,homeGoals,awayGoals,winner,date,weight
595,2026-01-17T15:00:00Z,22,FINISHED,Tottenham Hotspur FC,West Ham United FC,1,2,AWAY_TEAM,2026-01-17 15:00:00+00:00,1.993322
596,2026-01-17T17:30:00Z,22,FINISHED,Nottingham Forest FC,Arsenal FC,0,0,DRAW,2026-01-17 17:30:00+00:00,1.994992
597,2026-01-18T14:00:00Z,22,FINISHED,Wolverhampton Wanderers FC,Newcastle United FC,0,0,DRAW,2026-01-18 14:00:00+00:00,1.996661
598,2026-01-18T16:30:00Z,22,FINISHED,Aston Villa FC,Everton FC,0,1,AWAY_TEAM,2026-01-18 16:30:00+00:00,1.998331
599,2026-01-19T20:00:00Z,22,FINISHED,Brighton & Hove Albion FC,AFC Bournemouth,1,1,DRAW,2026-01-19 20:00:00+00:00,2.000000


In [176]:
# Compute home advantage
# Home advantage = average home goals - average away goals
home_avg = df_all["homeGoals"].mean()
away_avg = df_all["awayGoals"].mean()
home_advantage = home_avg - away_avg
home_advantage

0.19333333333333336

In [177]:
#  Calculate attack & defense strengths
teams = pd.unique(df_all[["homeTeam", "awayTeam"]].values.ravel("K"))

attack = pd.Series(1.0, index=teams)
defense = pd.Series(1.0, index=teams)

# Initialize with goals per match
team_stats = {}

for team in teams:
    home_games = df_all[df_all["homeTeam"] == team]
    away_games = df_all[df_all["awayTeam"] == team]

    goals_scored = (home_games["homeGoals"] * home_games["weight"]).sum() + \
                   (away_games["awayGoals"] * away_games["weight"]).sum()

    goals_against = (home_games["awayGoals"] * home_games["weight"]).sum() + \
                    (away_games["homeGoals"] * away_games["weight"]).sum()

    matches = home_games["weight"].sum() + away_games["weight"].sum()

    team_stats[team] = {
        "scored": goals_scored / matches,
        "against": goals_against / matches
    }

# Strengths = relative to league average
league_avg_scored = df_all["homeGoals"].mean() + df_all["awayGoals"].mean()
league_avg_scored /= 2

for team in teams:
    attack[team] = team_stats[team]["scored"] / league_avg_scored
    defense[team] = team_stats[team]["against"] / league_avg_scored

🔥 Summary

This function:
+ Calculates expected goals for each team
+ Uses Poisson distribution to compute goal probabilities
+ Converts score probabilities into match outcome probabilities
+ Returns probabilities for:
++ home win
++ draw
++ away win

The Poisson distribution models the number of goals a team scores in a match based on an expected goal rate (λ). Using the formula \(P(X=k)=e^{-\lambda}\lambda^k/k!\), it calculates the probability of scoring 0, 1, 2, … goals, where λ is estimated from team attack/defense strengths and league averages. In the model, I compute separate Poisson probabilities for home and away goals, then combine them to get the probabilities of each possible scoreline and therefore the probabilities of a home win, draw, or away win.


In [178]:
# Calculate probabilities for each future match

def match_probabilities(home, away):
    # expected goals
    exp_home = np.exp(np.log(league_avg_scored) + np.log(attack[home]) + np.log(defense[away]) + home_advantage)
    exp_away = np.exp(np.log(league_avg_scored) + np.log(attack[away]) + np.log(defense[home]))

    # compute probabilities up to 6 goals (I tested this number, and 6 produces the smallest rmse when compare to the bookmaker odds)
    max_goals = 6
    p_home = poisson.pmf(range(max_goals + 1), exp_home)
    p_away = poisson.pmf(range(max_goals + 1), exp_away)

    # result probabilities
    p_win = 0
    p_draw = 0
    p_loss = 0

    for i in range(max_goals + 1):
        for j in range(max_goals + 1):
            prob = p_home[i] * p_away[j]
            if i > j:
                p_win += prob
            elif i == j:
                p_draw += prob
            else:
                p_loss += prob

    return p_win, p_draw, p_loss

In [179]:
# Apply to all fixtures

results = []

for _, row in df_future.iterrows():
    home = row["homeTeam"]
    away = row["awayTeam"]

    p_win, p_draw, p_loss = match_probabilities(home, away)

    results.append({
        "utcDate": row["utcDate"],
        "homeTeam": home,
        "awayTeam": away,
        "p_home_win": p_win,
        "p_draw": p_draw,
        "p_away_win": p_loss,
    })

df_odds = pd.DataFrame(results)
df_odds.head()


,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win
0,2026-01-24T12:30:00Z,West Ham United FC,Sunderland AFC,0.300547,0.279822,0.419160
1,2026-01-24T15:00:00Z,Burnley FC,Tottenham Hotspur FC,0.264293,0.215537,0.514841
2,2026-01-24T15:00:00Z,Fulham FC,Brighton & Hove Albion FC,0.405496,0.228470,0.362613
3,2026-01-24T15:00:00Z,Manchester City FC,Wolverhampton Wanderers FC,0.775213,0.122029,0.071776
4,2026-01-24T17:30:00Z,AFC Bournemouth,Liverpool FC,0.304759,0.214773,0.474532


## 6. Compare calculated probabilities to bookmaker ones

In [180]:
unique_bet_home = betting_odds_avg["home_team"].unique()
unique_model_home = df_odds["homeTeam"].unique()

In [181]:
print(unique_bet_home)
print(unique_model_home)

['Newcastle United' 'Burnley' 'Brighton and Hove Albion'
 'Tottenham Hotspur' 'Crystal Palace' 'Sunderland' 'Arsenal' 'Bournemouth'
 'Brentford' 'Liverpool' 'Aston Villa' 'West Ham United' 'Chelsea'
 'Manchester City' 'Wolverhampton Wanderers' 'Nottingham Forest' 'Fulham'
 'Manchester United' 'Leeds United' 'Everton']
['West Ham United FC' 'Burnley FC' 'Fulham FC' 'Manchester City FC'
 'AFC Bournemouth' 'Crystal Palace FC' 'Brentford FC'
 'Newcastle United FC' 'Arsenal FC' 'Everton FC'
 'Brighton & Hove Albion FC' 'Leeds United FC'
 'Wolverhampton Wanderers FC' 'Chelsea FC' 'Liverpool FC' 'Aston Villa FC'
 'Manchester United FC' 'Nottingham Forest FC' 'Tottenham Hotspur FC'
 'Sunderland AFC']


In [182]:
def normalize_team(name):
    name = name.lower()
    name = name.replace(" fc", "")
    name = name.replace(" afc", "")
    name = name.replace("&", "and")
    name = name.replace("afc ", "")   # <--- this removes AFC from start
    name = name.strip()
    return name


In [183]:
df_odds["home_norm"] = df_odds["homeTeam"].apply(normalize_team)
df_odds["away_norm"] = df_odds["awayTeam"].apply(normalize_team)

betting_odds_avg["home_norm"] = betting_odds_avg["home_team"].apply(normalize_team)
betting_odds_avg["away_norm"] = betting_odds_avg["away_team"].apply(normalize_team)


In [184]:
unique_model_norm = df_odds["home_norm"].unique()
unique_bet_norm = betting_odds_avg["home_norm"].unique()

set(unique_model_norm) == set(unique_bet_norm)

True

In [185]:
df_compare = df_odds.merge(
    betting_odds_avg,
    left_on=["home_norm", "away_norm"],
    right_on=["home_norm", "away_norm"],
    how="inner"
)

print("Matched rows:", len(df_compare))
df_compare.head()

Matched rows: 20


,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win,home_norm,away_norm,home_team,away_team,p_home_book,p_draw_book,p_away_book
0,2026-01-24T12:30:00Z,West Ham United FC,Sunderland AFC,0.300547,0.279822,0.419160,west ham united,sunderland,West Ham United,Sunderland,0.380993,0.290701,0.328307
1,2026-01-24T15:00:00Z,Burnley FC,Tottenham Hotspur FC,0.264293,0.215537,0.514841,burnley,tottenham hotspur,Burnley,Tottenham Hotspur,0.256205,0.278063,0.465731
2,2026-01-24T15:00:00Z,Fulham FC,Brighton & Hove Albion FC,0.405496,0.228470,0.362613,fulham,brighton and hove albion,Fulham,Brighton and Hove Albion,0.376125,0.280316,0.343558
3,2026-01-24T15:00:00Z,Manchester City FC,Wolverhampton Wanderers FC,0.775213,0.122029,0.071776,manchester city,wolverhampton wanderers,Manchester City,Wolverhampton Wanderers,0.795081,0.135154,0.069765
4,2026-01-24T17:30:00Z,AFC Bournemouth,Liverpool FC,0.304759,0.214773,0.474532,bournemouth,liverpool,Bournemouth,Liverpool,0.257499,0.243565,0.498937


In [186]:
df_compare["diff_home"] = df_compare["p_home_win"] - df_compare["p_home_book"]
df_compare["diff_draw"] = df_compare["p_draw"] - df_compare["p_draw_book"]
df_compare["diff_away"] = df_compare["p_away_win"] - df_compare["p_away_book"]

df_compare[["homeTeam", "awayTeam", "diff_home", "diff_draw", "diff_away"]].head()

,homeTeam,awayTeam,diff_home,diff_draw,diff_away
0,West Ham United FC,Sunderland AFC,-0.080445,-0.010879,0.090853
1,Burnley FC,Tottenham Hotspur FC,0.008087,-0.062526,0.049110
2,Fulham FC,Brighton & Hove Albion FC,0.029370,-0.051846,0.019055
3,Manchester City FC,Wolverhampton Wanderers FC,-0.019868,-0.013125,0.002011
4,AFC Bournemouth,Liverpool FC,0.047260,-0.028791,-0.024404


In [187]:
rmse_home = np.sqrt(np.mean((df_compare["p_home_win"] - df_compare["p_home_book"])**2))
rmse_draw = np.sqrt(np.mean((df_compare["p_draw"] - df_compare["p_draw_book"])**2))
rmse_away = np.sqrt(np.mean((df_compare["p_away_win"] - df_compare["p_away_book"])**2))

rmse_home, rmse_draw, rmse_away

(0.05449905520163854, 0.03704891071844214, 0.047955009202484154)

In [188]:
rmse_total = np.sqrt(np.mean((
    df_compare["p_home_win"] - df_compare["p_home_book"]
)**2 + (
    df_compare["p_draw"] - df_compare["p_draw_book"]
)**2 + (
    df_compare["p_away_win"] - df_compare["p_away_book"]
)**2 ))

rmse_total

0.08150123748081793

> Note: RMSE_TOTAL varies often, as bookmaker odds vary. However, it's always stayed between 0.078 and 0.082.

In [189]:
np.mean([rmse_home, rmse_draw, rmse_away])

0.046500991707521616

In [190]:
df_compare["abs_diff"] = (
    abs(df_compare["diff_home"]) +
    abs(df_compare["diff_draw"]) +
    abs(df_compare["diff_away"])
)

df_compare.sort_values("abs_diff", ascending=False).head(10)[
    ["homeTeam", "awayTeam", "diff_home", "diff_draw", "diff_away"]
]


,homeTeam,awayTeam,diff_home,diff_draw,diff_away
16,Manchester United FC,Fulham FC,-0.117468,-0.000575,0.115498
8,Arsenal FC,Manchester United FC,0.107694,-0.061891,-0.053332
9,Everton FC,Leeds United FC,0.092096,-0.049729,-0.043742
0,West Ham United FC,Sunderland AFC,-0.080445,-0.010879,0.090853
6,Brentford FC,Nottingham Forest FC,0.064015,-0.060680,-0.007462
12,Wolverhampton Wanderers FC,AFC Bournemouth,-0.022945,-0.043591,0.062146
1,Burnley FC,Tottenham Hotspur FC,0.008087,-0.062526,0.049110
13,Chelsea FC,West Ham United FC,0.052959,-0.038334,-0.027373
19,Sunderland AFC,Burnley FC,0.056989,-0.027452,-0.031230
17,Nottingham Forest FC,Crystal Palace FC,-0.033333,-0.020426,0.053003


## 7. Replace my estimates probabilities with the ones I have from odds, creating my final match probabilities

In [191]:
df_odds.head(2)

,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win,home_norm,away_norm
0,2026-01-24T12:30:00Z,West Ham United FC,Sunderland AFC,0.300547,0.279822,0.419160,west ham united,sunderland
1,2026-01-24T15:00:00Z,Burnley FC,Tottenham Hotspur FC,0.264293,0.215537,0.514841,burnley,tottenham hotspur


In [192]:
betting_odds_avg.head(2)

,home_team,away_team,p_home_book,p_draw_book,p_away_book,home_norm,away_norm
0,Newcastle United,Aston Villa,0.480580,0.256018,0.263401,newcastle united,aston villa
1,Burnley,Tottenham Hotspur,0.256205,0.278063,0.465731,burnley,tottenham hotspur


In [193]:
df_final_probabilities = df_odds.merge(
    betting_odds_avg,
    left_on=["home_norm", "away_norm"],
    right_on=["home_norm", "away_norm"],
    how="left"
)

In [194]:
df_final_probabilities = df_final_probabilities[[
    "utcDate",
    "homeTeam",
    "awayTeam",
    "p_home_win",
    "p_draw",
    "p_away_win",
    "p_home_book",
    "p_draw_book",
    "p_away_book",
]]

df_final_probabilities

,utcDate,homeTeam,awayTeam,p_home_win,p_draw,p_away_win,p_home_book,p_draw_book,p_away_book
0,2026-01-24T12:30:00Z,West Ham United FC,Sunderland AFC,0.300547,0.279822,0.419160,0.380993,0.290701,0.328307
1,2026-01-24T15:00:00Z,Burnley FC,Tottenham Hotspur FC,0.264293,0.215537,0.514841,0.256205,0.278063,0.465731
2,2026-01-24T15:00:00Z,Fulham FC,Brighton & Hove Albion FC,0.405496,0.228470,0.362613,0.376125,0.280316,0.343558
3,2026-01-24T15:00:00Z,Manchester City FC,Wolverhampton Wanderers FC,0.775213,0.122029,0.071776,0.795081,0.135154,0.069765
4,2026-01-24T17:30:00Z,AFC Bournemouth,Liverpool FC,0.304759,0.214773,0.474532,0.257499,0.243565,0.498937
...,...,...,...,...,...,...,...,...,...
155,2026-05-24T15:00:00Z,Liverpool FC,Brentford FC,0.564764,0.197342,0.228107,NaN,NaN,NaN
156,2026-05-24T15:00:00Z,Manchester City FC,Aston Villa FC,0.582085,0.210743,0.201947,NaN,NaN,NaN
157,2026-05-24T15:00:00Z,Nottingham Forest FC,AFC Bournemouth,0.411739,0.237440,0.348489,NaN,NaN,NaN
158,2026-05-24T15:00:00Z,Tottenham Hotspur FC,Everton FC,0.420254,0.258064,0.320653,NaN,NaN,NaN


In [195]:
df_final_probabilities["p_home_final"] = np.where(
    df_final_probabilities["p_home_book"].notna(),
    df_final_probabilities["p_home_book"],
    df_final_probabilities["p_home_win"]
)

df_final_probabilities["p_draw_final"] = np.where(
    df_final_probabilities["p_draw_book"].notna(),
    df_final_probabilities["p_draw_book"],
    df_final_probabilities["p_draw"]
)

df_final_probabilities["p_away_final"] = np.where(
    df_final_probabilities["p_away_book"].notna(),
    df_final_probabilities["p_away_book"],
    df_final_probabilities["p_away_win"]
)

In [196]:
print("Used betting odds:", df_final_probabilities["p_home_book"].notna().sum())
print("Used model:", df_final_probabilities["p_home_book"].isna().sum())


Used betting odds: 20
Used model: 140


In [197]:
df_final_probabilities = df_final_probabilities[[
    "utcDate",
    "homeTeam",
    "awayTeam",
    "p_home_final",
    "p_draw_final",
    "p_away_final"
]]

In [198]:
df_final_probabilities

,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final
0,2026-01-24T12:30:00Z,West Ham United FC,Sunderland AFC,0.380993,0.290701,0.328307
1,2026-01-24T15:00:00Z,Burnley FC,Tottenham Hotspur FC,0.256205,0.278063,0.465731
2,2026-01-24T15:00:00Z,Fulham FC,Brighton & Hove Albion FC,0.376125,0.280316,0.343558
3,2026-01-24T15:00:00Z,Manchester City FC,Wolverhampton Wanderers FC,0.795081,0.135154,0.069765
4,2026-01-24T17:30:00Z,AFC Bournemouth,Liverpool FC,0.257499,0.243565,0.498937
...,...,...,...,...,...,...
155,2026-05-24T15:00:00Z,Liverpool FC,Brentford FC,0.564764,0.197342,0.228107
156,2026-05-24T15:00:00Z,Manchester City FC,Aston Villa FC,0.582085,0.210743,0.201947
157,2026-05-24T15:00:00Z,Nottingham Forest FC,AFC Bournemouth,0.411739,0.237440,0.348489
158,2026-05-24T15:00:00Z,Tottenham Hotspur FC,Everton FC,0.420254,0.258064,0.320653


In [199]:
df_final_probabilities["homeTeam"].unique()

array(['West Ham United FC', 'Burnley FC', 'Fulham FC',
       'Manchester City FC', 'AFC Bournemouth', 'Crystal Palace FC',
       'Brentford FC', 'Newcastle United FC', 'Arsenal FC', 'Everton FC',
       'Brighton & Hove Albion FC', 'Leeds United FC',
       'Wolverhampton Wanderers FC', 'Chelsea FC', 'Liverpool FC',
       'Aston Villa FC', 'Manchester United FC', 'Nottingham Forest FC',
       'Tottenham Hotspur FC', 'Sunderland AFC'], dtype=object)

In [200]:
name_map = {
    "Aston Villa FC": "Aston Villa",
    "Brighton & Hove Albion FC": "Brighton & Hove Albion",
    "AFC Bournemouth": "AFC Bournemouth",   # keep as is
    "Bournemouth": "AFC Bournemouth",
    "Sunderland AFC": "Sunderland",
    "Newcastle United FC": "Newcastle United",
    "Manchester City FC": "Manchester City",
    "Manchester United FC": "Manchester United",
    "West Ham United FC": "West Ham United",
    "Wolverhampton Wanderers FC": "Wolverhampton Wanderers",
    "Tottenham Hotspur FC": "Tottenham Hotspur",
    "Crystal Palace FC": "Crystal Palace",
    "Brentford FC": "Brentford",
    "Everton FC": "Everton",
    "Leeds United FC": "Leeds United",
    "Chelsea FC": "Chelsea",
    "Liverpool FC": "Liverpool",
    "Nottingham Forest FC": "Nottingham Forest",
    "Burnley FC": "Burnley",
    "Fulham FC": "Fulham",
    "Arsenal FC": "Arsenal"
}

df_final_probabilities["home_team_norm"] = df_final_probabilities["homeTeam"].replace(name_map)
df_final_probabilities["away_team_norm"] = df_final_probabilities["awayTeam"].replace(name_map)

premierleague["team_norm"] = premierleague["team"].replace({
    "Brighton & Hove Albion": "Brighton & Hove Albion",
    "AFC Bournemouth": "AFC Bournemouth"
})


[01/19/26 20:49:54] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_21604\2477332962.py:25:    ]8;id=39825;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=922194;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_final_probabilities["home_team_norm"] =                                          
                             df_final_probabilities["homeTeam"].replace(name_map)                                  
                                                                                                                   

                    WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_21604\2477332962.py:26:    ]8;id=332584;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=797714;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             SettingWithCopyWarning:                                                               
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_final_probabilities["away_team_norm"] =                                          
                             df_final_probabilities["awayTeam"].replace(name_map)                                  
                                                                                                                   

In [201]:
df_simulation = df_final_probabilities.copy()

In [202]:
# Normalize probabilities so they sum to 1
prob_cols = ["p_home_final", "p_draw_final", "p_away_final"]
df_simulation[prob_cols] = df_simulation[prob_cols].div(df_simulation[prob_cols].sum(axis=1), axis=0)

In [203]:
df_simulation.head()

,utcDate,homeTeam,awayTeam,p_home_final,p_draw_final,p_away_final,home_team_norm,away_team_norm
0,2026-01-24T12:30:00Z,West Ham United FC,Sunderland AFC,0.380993,0.290701,0.328307,West Ham United,Sunderland
1,2026-01-24T15:00:00Z,Burnley FC,Tottenham Hotspur FC,0.256205,0.278063,0.465731,Burnley,Tottenham Hotspur
2,2026-01-24T15:00:00Z,Fulham FC,Brighton & Hove Albion FC,0.376125,0.280316,0.343558,Fulham,Brighton & Hove Albion
3,2026-01-24T15:00:00Z,Manchester City FC,Wolverhampton Wanderers FC,0.795081,0.135154,0.069765,Manchester City,Wolverhampton Wanderers
4,2026-01-24T17:30:00Z,AFC Bournemouth,Liverpool FC,0.257499,0.243565,0.498937,AFC Bournemouth,Liverpool


## 8. Run simulations to build the Premier League table probabilities

In [204]:
def simulate_once(fixtures, table):
    table_sim = table.copy()

    # Use normalized team name column
    points = dict(zip(table_sim["team_norm"], table_sim["pts"]))

    for _, row in fixtures.iterrows():
        home = row["home_team_norm"]
        away = row["away_team_norm"]

        # choose outcome
        probs = [row["p_home_final"], row["p_draw_final"], row["p_away_final"]]
        outcome = np.random.choice(["H", "D", "A"], p=probs)

        if outcome == "H":
            points[home] += 3
        elif outcome == "D":
            points[home] += 1
            points[away] += 1
        else:
            points[away] += 3

    result_df = table_sim.copy()
    result_df["pts"] = result_df["team_norm"].map(points)

    # sort by points and goal difference
    result_df = result_df.sort_values(["pts", "gd"], ascending=[False, False])
    result_df["position"] = np.arange(1, len(result_df)+1)

    return result_df


def run_simulations(fixtures, table, n_sim=10000):
    position_counts = {team: np.zeros(len(table)) for team in table["team_norm"]}

    for _ in range(n_sim):
        final_table = simulate_once(fixtures, table)

        for _, row in final_table.iterrows():
            position_counts[row["team_norm"]][row["position"]-1] += 1

    pos_df = pd.DataFrame(position_counts, index=np.arange(1, len(table)+1))
    pos_df.index.name = "position"
    return pos_df

In [205]:
# RUN
position_distribution = run_simulations(df_simulation, premierleague, n_sim=20000)

In [206]:
position_distribution.index.name = "TEAM"
position_distribution_t = position_distribution.T

In [207]:
position_distribution_pct = position_distribution_t.div(
    position_distribution_t.sum(axis=1),
    axis=0
) * 100


## 9. Preview and present the results graphically

In [208]:
# Build label mapping: "position  team" (extra space for 1-9)
team_labels = (
    premierleague[["team", "position"]]
    .set_index("team")["position"]
    .map(lambda pos: f"{pos}{'  ' if pos < 10 else ' '}")
)

# Join position and team name into one label
team_labels = (
    premierleague[["team", "position"]]
    .assign(
        label=lambda df: df.apply(
            lambda r: f"{r['position']}{'&nbsp;&nbsp;&nbsp;&nbsp;' if r['position'] < 10 else '&nbsp;&nbsp;'}{r['team']}",
            axis=1
        )
    )
    .set_index("team")["label"]
)


# Apply labels to your table index
position_distribution_pct.index = position_distribution_pct.index.map(team_labels)

# Drop position column if present
position_distribution_pct = position_distribution_pct.drop(columns=["position"], errors="ignore")

# Remove index name
position_distribution_pct.index.name = None


In [209]:
position_distribution_pct.style\
    .background_gradient(
        cmap=green_cmap,
        vmin=0,
        vmax=vmax
    )\
    .applymap(zero_style)\
    .format("{:.2f}%")\
    .set_table_styles([
        {"selector": "th", "props": [
            ("background-color", "#e6edf4"),
            ("color", "#333"),
            ("text-align", "center"),
            ("font-family", "Inter, Roboto, Arial, sans-serif"),
            ("font-size", "13px"),
            ("font-weight", "600")
        ]},

        # <-- LEFT align only the "position" column header
        {"selector": "th.col_heading", "props": [
            ("text-align", "centre")
        ]},


        {"selector": "th.row_heading", "props": [
            ("text-align", "left"),
            ("font-size", "13px"),
            ("font-weight", "600"),
            ("white-space", "nowrap"),
            ("max-width", "250px"),
            ("overflow", "hidden"),
            ("text-overflow", "ellipsis")
        ]},

        {"selector": "tr:nth-child(odd) th.row_heading", "props": [
            ("background-color", "#fbfcfe")
        ]},
        {"selector": "tr:nth-child(even) th.row_heading", "props": [
            ("background-color", "#e6edf4")
        ]},

        {"selector": "td", "props": [
            ("text-align", "center"),
            ("font-family", "Inter, Roboto, Arial, sans-serif"),
            ("font-size", "12px"),
            ("font-weight", "500"),
            ("color", "#000")
        ]}
    ])


[01/19/26 20:54:58] WARNING  C:\Users\vicky\AppData\Local\Temp\ipykernel_21604\3212083147.py:7:     ]8;id=763268;file://C:\ProgramData\anaconda3\Lib\warnings.py\warnings.py]8;;\:]8;id=876428;file://C:\ProgramData\anaconda3\Lib\warnings.py#112\112]8;;\
                             FutureWarning: Styler.applymap has been deprecated. Use Styler.map                    
                             instead.                                                                              
                               .applymap(zero_style)\                                                              
                                                                                                                   

TEAM,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1 Arsenal,89.94%,9.10%,0.87%,0.08%,0.01%,0.01%,0.00%,0.01%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2 Manchester City,8.91%,65.41%,18.18%,5.17%,1.65%,0.48%,0.15%,0.03%,0.01%,0.01%,0.00%,0.01%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
3 Aston Villa,0.92%,16.96%,41.91%,21.91%,10.10%,4.40%,1.98%,1.05%,0.46%,0.18%,0.05%,0.06%,0.01%,0.01%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
4 Liverpool,0.21%,6.22%,21.77%,29.33%,17.88%,11.24%,5.85%,3.33%,2.01%,1.18%,0.50%,0.27%,0.12%,0.06%,0.03%,0.01%,0.00%,0.00%,0.00%,0.00%
5 Manchester United,0.00%,0.10%,1.44%,4.98%,9.37%,12.63%,14.24%,13.45%,11.46%,9.28%,7.52%,6.01%,4.49%,2.76%,1.42%,0.71%,0.14%,0.00%,0.00%,0.00%
6 Chelsea,0.02%,1.54%,10.02%,19.71%,22.88%,15.97%,11.12%,7.21%,4.61%,3.02%,1.76%,0.99%,0.61%,0.40%,0.08%,0.04%,0.01%,0.00%,0.00%,0.00%
7 Brentford,0.00%,0.25%,1.87%,6.14%,11.41%,15.11%,14.88%,13.25%,10.90%,8.56%,6.44%,4.70%,3.05%,1.85%,1.00%,0.45%,0.16%,0.01%,0.00%,0.00%
8 Newcastle United,0.01%,0.35%,2.73%,7.56%,13.08%,15.58%,14.70%,13.03%,10.21%,7.91%,5.65%,4.09%,2.48%,1.39%,0.84%,0.30%,0.08%,0.00%,0.00%,0.00%
9 Sunderland,0.00%,0.03%,0.45%,1.77%,4.26%,7.02%,9.90%,11.60%,12.35%,12.53%,11.65%,9.75%,7.49%,5.30%,3.46%,1.77%,0.64%,0.03%,0.00%,0.00%
10 Everton,0.00%,0.01%,0.19%,0.92%,2.33%,4.14%,6.50%,8.22%,10.78%,11.96%,12.43%,12.07%,10.68%,8.75%,6.12%,3.50%,1.34%,0.06%,0.00%,0.00%
